In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import randint, uniform

In [2]:
df = pd.read_csv('data.csv')
df.head()

,Player,Season,Passing Yds,Passing Tds,Rushing Yds,Rushing Tds,Receiving Yds,Receiving Tds,VBD
0,Ron Johnson,1970,0,0,1027,8,48,5,135
1,Gene Washington,1970,0,0,0,0,44,1,33
2,MacArthur Lane,1970,0,0,977,11,32,2,126
3,Warren Wells,1970,0,0,34,0,43,0,112
4,John Brodie,1970,2941,24,29,2,0,1,105


In [3]:
aggregated = df.groupby("Player").agg({
    'Passing Yds': 'sum',
    'Passing Tds': 'sum',
    'Rushing Yds': 'sum',
    'Rushing Tds': 'sum',
    'Receiving Yds': 'sum',
    'Receiving Tds': 'sum',
    'VBD': 'mean'  # Keep target (VBD) as the mean for regression
}).reset_index()

In [4]:
# Features to normalize
features = ['Passing Yds', 'Passing Tds', 'Rushing Yds', 'Rushing Tds', 'Receiving Yds', 'Receiving Tds']

scaler = MinMaxScaler()
aggregated[features] = scaler.fit_transform(aggregated[features])

In [5]:
X = aggregated.drop(['Player', 'VBD'], axis=1)
y = aggregated['VBD']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
rf = RandomForestRegressor(random_state=42)

param_dist = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'max_features': uniform(0.5, 0.5)  # try between 0.5 and 1.0
}

search = RandomizedSearchCV(
    rf,
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_mean_squared_error',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

search.fit(X_train, y_train)

print('Best hyperparameters:', search.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best hyperparameters: {'max_depth': 11, 'max_features': 0.6394356762960909, 'min_samples_leaf': 3, 'min_samples_split': 9, 'n_estimators': 307}


In [7]:
best_rf = search.best_estimator_
y_pred = best_rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)

print(f'\nTest set performance:')
print(f'  MSE : {mse:.2f}')
print(f'  RMSE: {rmse:.2f}')
print(f'  MAE : {mae:.2f}')


Test set performance:
  MSE : 33.03
  RMSE: 5.75
  MAE : 2.06


In [8]:
# Cross‑validated RMSE on the full training set
cv_scores = cross_val_score(best_rf, X_train, y_train, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
cv_rmse = np.sqrt(-cv_scores)
print(f'5‑fold CV RMSE: {cv_rmse.mean():.2f} ± {cv_rmse.std():.2f}')

5‑fold CV RMSE: 6.11 ± 0.38


In [9]:
# Refit on entire dataset and assign scores
best_rf.fit(X, y)

RandomForestRegressor(max_depth=11, max_features=0.6394356762960909,
                      min_samples_leaf=3, min_samples_split=9, n_estimators=307,
                      random_state=42)

In [10]:
aggregated["Predicted_Score"] = best_rf.predict(X)

top10 = aggregated[['Player', 'Predicted_Score', 'VBD']].sort_values('Predicted_Score', ascending=False).head(10)
print('\nTop 10 Players by Predicted Score:')
print(top10)


Top 10 Players by Predicted Score:
                   Player  Predicted_Score         VBD
2172         Emmitt Smith        87.241573   86.933333
4024  LaDainian Tomlinson        85.831276  118.181818
372         Barry Sanders        85.132778  120.700000
6643        Walter Payton        84.864588   98.846154
4450       Marshall Faulk        77.974343   96.083333
1350        Curtis Martin        75.149264   76.818182
4337         Marcus Allen        73.795629   54.937500
5819      Shaun Alexander        66.353474   81.555556
1809        Derrick Henry        65.966732   82.222222
2286           Frank Gore        64.680120   36.250000
